# Classify images using GoogleNet
Helper notebook used to classify the noisy images using GoogleNet (the same net used in our TCAV).
Gives accuracies and the largest classes - used for plotting in our results.

In [78]:
import os
import torch
import json
import numpy as np

from PIL import Image
from googlenet_pytorch import GoogLeNet
import torchvision.transforms as transforms

import csv
# torch.device("mps")

In [79]:
# læs labels
labels_map = [i for i in open("labels_map.txt").read().splitlines()]
model = GoogLeNet.from_pretrained('googlenet')
model.eval()

Loaded pretrained weights for googlenet


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [80]:
# Preprocess image
preprocess = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [81]:
def path_to_tensor(path):
    input_image = Image.open(path).convert('RGB')

    input_tensor = preprocess(input_image)
    return input_tensor.unsqueeze(0)

In [82]:

results = []

INPUT = "zebra_noise90pct"
input_path = os.getcwd() + "/data/" + INPUT + "/"
output_path = os.getcwd() + "/data/zebra_preds/" + INPUT + ".csv"

for i, filename in enumerate(os.listdir(input_path)):
    input_batch = path_to_tensor(input_path + filename)

    with torch.no_grad():
        logits = model(input_batch)

    preds = torch.topk(logits, k=1).indices.squeeze(0).tolist()
    
    for idx in preds:
        label = labels_map[idx]
        prob = torch.softmax(logits, dim=1)[0, idx].item()
        results.append([filename, label, prob])
        print(f"{filename}:  {label:<75} ({prob * 100:.2f}%)")


# Write predicitions to dataframe
with open(output_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(results)

63.jpg:  prayer rug, prayer mat                                                      (15.75%)
77.jpg:  dishrag, dishcloth                                                          (3.22%)
88.jpg:  prayer rug, prayer mat                                                      (20.78%)
89.jpg:  window screen                                                               (14.90%)
76.jpg:  dishrag, dishcloth                                                          (3.69%)
62.jpg:  dishrag, dishcloth                                                          (13.18%)
74.jpg:  window screen                                                               (3.67%)
60.jpg:  doormat, welcome mat                                                        (10.80%)
48.jpg:  doormat, welcome mat                                                        (3.46%)
49.jpg:  prayer rug, prayer mat                                                      (18.74%)
61.jpg:  prayer rug, prayer mat                                 